# Generating puzzles with multiple solutions
Puzzles with multiple solutions can create very interesting effects, such as the puzzle shown in the video "[How can a jigsaw have two distinct solutions?](https://www.youtube.com/watch?v=b5nElEbbnfU)" on the Stand-Up Maths Youtube channel where the two ways of solving the jigsaw puzzle either create an image of a coffee cup or a donut. This page describes how such puzzles can be generated by first picking a random scrambling of the puzzle pieces, and then generating a puzzle that is solvable in both its unscrambled and scrambled state. There is also Python code.

# Describing puzzles using vectors
In order for a jigsaw puzzle to be solvable in more than one way, there must be several connections between pieces that have the same shape. Otherwise, if every connection was unique, each side of a piece would only be able to connect with a single side of a single other piece, and there would only be a single solution to the puzzle. This means that each side of each puzzle piece can be grouped with other puzzle piece sides with a similar shape, and these groups can be given numbers.

It is useful to give the shapes numbers so that shapes that are compatible with each other have inverse numbers. For example, shape 1 is compatible with shape -1, shape 2 is compatible with shape -2, and so forth. This makes it easy to check whether two shapes are compatible: they are compatible if they add up to 0, and incompatible if they do not. I prefer to think of the positive numbers as all the shapes that stick out and the negative numbers as the indented shapes, but this is fully arbitrary. Finally, the outer sides of the pieces at the edge of the jigsaw puzzle are just shaped like straight lines, and do not connect to anything. These non-connecting shapes are labelled 0.

A puzzle can be described mathematically as a vector containing the shapes of each side of all the pieces of the puzzle. The order of the sides is also arbitrary, but here we order the sides in order left, top, right, bottom, starting with the top left piece and going row by row. The ordering of sides for a 2x2 puzzle is illustrated in the following figure.

<img src="img/side_numbers.png" style="width:500px; margin-left:auto; margin-right:auto;"/>

Thus, if we assume that all the tabs in the image are in shape group 1 and all the indentations are in shape group -1, the vector describing this puzzle is:
$$
    \vec{p} = \begin{pmatrix}
        0 \\
        0 \\
        -1 \\
        1 \\
        1 \\
        0 \\
        0 \\
        1 \\
        0 \\
        -1 \\
        -1 \\
        0 \\
        1 \\
        -1 \\
        0 \\
        0
    \end{pmatrix}
$$

# Scramble matrices
In order to solve a puzzle, the pieces have to be moved around and rotated. I will refer to a series of movements and rotations as a scramble. These scrambles can be represented as matrices, which can be applied to the puzzle vector. A scramble matrix can be constructed as follows:
* Start with an $n\times n$ matrix with all entries set to 0, where n is the number of puzzle piece sides (and thus the length of vector $\vec{p}$).
* For each side in the puzzle, find its original position $a$ and its scrambled position $b$. Set the entry at row $b$ and column $a$ to 1.

There are a few constraints to this scramble matrix. Each row and each column in the matrix can only have a single entry that is equal to one, since all sides in the original puzzle have to be present in the scrambled puzzle, and sides can not be duplicated or destroyed. It is also not possible to move sides independently, since they are connected to puzzle pieces. This means that each $4\times4$ section of the scramble matrix can only take one of the following forms:

$$
    \mathbf{R}_0 = \begin{pmatrix}
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 1
    \end{pmatrix},
    \mathbf{R}_1 = \begin{pmatrix}
        0 & 1 & 0 & 0 \\
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 1 \\
        1 & 0 & 0 & 0
    \end{pmatrix},
    \mathbf{R}_2 = \begin{pmatrix}
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 1 \\
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0
    \end{pmatrix},
    \mathbf{R}_3 = \begin{pmatrix}
        0 & 0 & 0 & 1 \\
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 1 & 0
    \end{pmatrix},
    \mathbf{\phi} = \begin{pmatrix}
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0
    \end{pmatrix}
$$
If the submatrix starting at row $4i$ and column $4j$ in the scramble matrix looks like $\mathbf{R}_0$, then the scramble involves moving piece number $j$ to the position of piece number $i$ without rotating it. If the submatrix looks like $\mathbf{R}_1$ then piece $j$ was moved to the position of piece $i$ and rotated once counter-clockwise. Likewise, $\mathbf{R}_2$ means the piece has been moved and rotated twice, while $\mathbf{R}_3$ means the piece has been moved and rotated counter-clockwise three times (equivalent to rotating clockwise once). Submatrix $\mathbf{\phi}$ means that piece $j$ was not moved to the position of piece $i$.

As an example, the following scramble matrix involves rotating the first piece of a $2\times2$ puzzle once counter-clockwise, and then swapping the positions of the first and fourth pieces.
$$
    \mathbf{T} = \begin{pmatrix}
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
        0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
    \end{pmatrix}
$$

# Validating if a puzzle is solved
A puzzle can be validated by checking each side of the puzzle: if the side is on an outside edge of the puzzle, it must have a connection shape of 0, while if the side faces another side in the puzzle, the sum of the shapes of the two sides must be 0. This set of checks can be encoded as a verification matrix, such that a puzzle is solved if and only if its puzzle vector $\vec{p}$ satisfies the equation
$$
    \mathbf{V}\vec{p} = \vec{0}
$$
Each row in the verification matrix corresponds to checking a single side in the puzzle. If that side faces another side in the puzzle, the the column for each of those sides is set to one. If the side is on the outside edge of the puzzle, then only the column corresponding to that side is set to one. Each row that corresponds to checking two sides that face each other is repeated twice, since the check is made once for each side in the pair. It is useful to order the rows so that row number $i$ corresponds to checking if side number $i$ is valid, since this will make each entry on the main diagonal of the matrix have the value one, and will also make the matrix symmetric about the main diagonal.

It is notable that since $\mathbf{V}\vec{p} = \vec{0}$ has many non-trivial solutions for $\vec{p}$ (any valid puzzle is a solution), the verification matrix $\mathbf{V}$ must be non-invertible.

As an example, the verification matrix for a 2x2 puzzle looks like this:
$$
    \mathbf{V} = \begin{pmatrix}
        1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
    \end{pmatrix}
$$

In [ ]:
%load src/verification_matrix.py

 # Finding puzzles with multiple solutions
 Any arbitrary scramble (i.e. movement and/or rotation of the pieces) of a puzzle can be described using a scramble matrix, $\mathbf{T}$. If the matrix equation $\mathbf{V}\mathbf{T}\vec{p} = \vec{0}$ holds, then this scramble is a solution to the puzzle. For a given valid scramble (more on that in the next section) $\mathbf{T}$ and a given verification matrix $\mathbf{V}$, it is possible to construct a puzzle vector $\vec{p}$ such that the following system of matrix equations hold true:
 $$
    \mathbf{V}\mathbf{T}\vec{p} = \vec{0}
 $$
 $$
    \mathbf{V}\vec{p} = \vec{0}
 $$

A puzzle $\vec{p}$ that satisfies these constraints must have at least two solutions, since both its unscrambled state and its state that is scrambled according to $\mathbf{T}$ are solutions.

In [ ]:
%load src/puzzle_generator.py

# Random scrambles and scramble similarity
Since we now know how to generate a puzzle that has at least two solutions given an arbitrary scramble, we can generate a random scramble and build a puzzle with multiple solutions using that. However, it is important to keep in mind that the scramble can't be completely arbitrary: only corner pieces can occupy the corners of the jigsaw puzzle, and only edge pieces can occupy the edges, and their orientation is determined by which edge they're on. All middle pieces can be arbitrarily rearranged and rotated. Any scramble that conforms to these rules is valid, and can be used to construct a puzzle vector $\vec{p}$.

In the video, Matt Parker also wants the two solutions to a puzzle to be as different as possible. One way of quantifying how similar the puzzle is after being scrambled is to count how many sides are adjacent to each other in the scrambled puzzle that were also adjacent in the original puzzle. This is easily countable: if every connecting shape in the original puzzle is unique, then every pair of sides that is still able to connect after scrambling the puzzle are sides that were connected in the original puzzle. This means that if a puzzle vector $\vec{p}_u$ is constructed so that no two connections have the same shape, the number of edges in the puzzle is denoted as $n_\text{edges}$, and $n_\text{non-zero}$ is equal to the number of non-zero entries in the vector $\mathbf{V}\mathbf{T}\vec{p}_u$, the similarity of the puzzle after being scrambled is equal to
$$
    s = \frac{n_\text{non-zero}-n_\text{edges}}{2}
$$

In [ ]:
%load src/scrambles.py

# Counting the number of solutions
In order to achieve the effect in the video where the puzzle motif changes from a coffee cup to a donut, the puzzle should only have two solutions, otherwise it would be possible to put it back together in different ways that create nonsensical motifs. Since the above method guarantees that a puzzle has *at least* two solutions, we need to count the number of possible solutions to ensure that the puzzle has *only* two solutions.

Counting the number of possible solutions amounts to finding the number of matrices $\mathbf{T}$ that satisfies the equation
$$
    \mathbf{V}\mathbf{T}\vec{p} = \vec{0}
$$
where $\mathbf{V}$ and $\vec{p}$ are known. I don't think there exists a better way of finding the number of solutions than trial and error. My explanation for this is given in the next cell in this notebook, which I have hidden by default.

Since counting the number of solutions takes a long time, it is useful to weed out puzzles that obviously have more than two solutions. Any where two pieces have the exact same side shapes must have at least two solutions, since we can solve the puzzle by simply swapping those two pieces. Since we already know that our chosen puzzle also has a solution that is more complicated than just swapping two pieces, our puzzle must have more than two solutions.

It also seems likely that the more often a particular connection shape is repeated, the more likely a puzzle is to have many solutions. Additionally, puzzles with fewer repeated connections should be faster to check, and Matt Parker also states in the video that the most satisfying puzzle would be one with only two solutions, no shared sides, and no more than two of any connection type. This seems almost impossible to find, but setting an upper bound on the number of repeated connection shapes will also weed out a lot of puzzles.

# Attempt at finding $\mathbf{T}$ through linear algebra
In order to express our constraints on the entries of the scramble matrix $\mathbf{T}$, we can unroll its entries, row by row, into a scramble vector $\vec{t}$ so that the entries of $\vec{t}$ look like:
$$
    \vec{t} = \begin{pmatrix}
        T_{1,1} \\
        T_{1,2} \\
        \vdots \\
        T_{1,n} \\
        T_{2,1} \\
        \vdots \\
        T_{3,1} \\
        \vdots \\
        T_{n,n}
    \end{pmatrix}
$$
We can encode our constraints into a matrix $\mathbf{C}$, such that $\mathbf{T}$ is only a valid scramble matrix if its entry vector $\vec{t}$ satisfies the following equation:
$$
    \mathbf{C}\vec{t}=\vec{b}
$$
I'll illustrate what constraints can be encoded into $\mathbf{C}$ and $\vec{b}$ in the next section.

We already know that $\mathbf{C}\vec{t}=\vec{b}$ must be an underdetermined system, since there exists several scrambles that solves the puzzle. This means that $\mathbf{C}$ has no inverse. However, $\mathbf{C}$ must have a unique pseudo-inverse $\mathbf{C}^\dagger$, and using this we can determine that every solution of $\vec{t}$ must lie in the space 
$$
    \vec{t} = \mathbf{C}^\dagger\vec{b} + \left[\mathbb{I} - \mathbf{C}^\dagger\mathbf{C}\right]\vec{\omega}
$$
where $\vec{\omega}$ is an arbitrary vector (see [the article on the Moore-Penrose inverse on Wikipedia](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse#Applications)). However, even though $\vec{t}$ has infinitely many solutions, we are only looking for the ones where all the values of $\vec{t}$ are either $0$ or $1$. I am really not an expert on multidimensional geometry, but I suspect that is is possible to define some other subspaces where the entries of $\vec{t}$ are locked to values that correspond to putting certain pieces down, and then checking whether this space of possible scramble vectors $\vec{t}$ and the solution space intersect. I also suspect that this is a very difficult and computationally expensive operation, but, again, I am really not an expert on multidimensional geometry. Worst case, one could find the points in the vector space of $\vec{t}$ that corresponds to every single permutation of the puzzle, and check which of those points lie inside the solution space, but this would entail checking every single possible permutation of all the puzzle pieces, and would be way worse than finding the solution through traditional trial and error.

Another problem with the linear algebra method of solving for the scramble matrix is the size of the data structures involved. Since the scramble matrix has dimensions $4n\times 4n$, where $n$ is the number of pieces in the puzzle, the scramble vector has dimensions $16n^2\times1$. This again means that the matrix $\mathbf{C}^\dagger\mathbf{C}$ will have dimensions of $16n^2\times16n^2$, which is getting prohibitively large.

## An estimate on how many pieces it is necessary to place before the scramble matrix can be determined
As mentioned previously, it is possible to encode constraints on the scramble matrix so that every valid scramble matrix must satisfy the equation
$$
    \mathbf{C}\vec{t}=\vec{b}
$$
As also mentioned previously, as long as this system of equations is underdetermined, there also exists an infinity of invalid scramble matrices that satisfy the equation. In order for the system of equations above to have a single solution, the number of linearly independent equations encoded in the constraint matrix $\mathbf{C}$ must be equal to or larger than the number of entries in the scramble vector $\vec{t}$. Some constraints follow from the fact that we want the scramble matrix to be valid and the puzzle to be solved, while additional constraints can be added to the system by locking puzzle pieces into place in the scramble.

#### Constraints due to connection shapes
Before any pieces are placed, there are already some constraints on the scramble matrix. Any valid scramble matrix must move the pieces so that only sides with compatible shapes end up next to each other. For example, if entry number 2 and 4 of the puzzle vector represent sides that are adjacent to each other in the assembled puzzle, rows 2 and 4 in the scramble matrix are both constrained by the equation
$$
    (T_{2,1} + T_{4,1})p_1 + (T_{2,2} + T_{4,2})p_2 + \dots + (T_{2,n} + T_{4,n})p_n = 0
$$
This can be encoded in the constraint matrix as
$$
    \begin{pmatrix}
        \dots & 0 & p_1 & \dots & p_n & 0 & \dots & 0 & p_1 & \dots & p_n & 0 & \dots
    \end{pmatrix}
    \begin{pmatrix}
        \vdots \\ T_{1,n} \\ T_{2,1} \\ \vdots \\ T_{2,n} \\ T_{3,1} \\ \vdots \\ T_{3,n} \\ T_{4,1} \\ \vdots \\ T_{4,n} \\ T_{5,1} \\ \vdots
    \end{pmatrix}
    = \begin{pmatrix}
        0
    \end{pmatrix}
$$
Since constraints need to be linearly independent in order to contribute to a solution, it is not useful to add more than one constraint for each pair of sides facing each other. The constraints on the outside edges of the puzzle are also not linearly independent of these constraints: all the non-zero entries in the puzzle vector must be used to satisfy these constraints, meaning that the outside edges must automatically all have the value 0 if the set of constraints is satisfied.

If the width of the puzzle is $w$ and the height is $h$, there are $2wh - w - h$ inside connections in the puzzle, meaning we can construct $2wh - w -h$ rows in the constraint matrix.

#### Constraints on the rows and columns of the scramble matrix
The entries in the scramble matrix cannot be completely arbitrary. As mentioned previously each row and each column can only have a single entry with the value of one. This can be encoded in the constraint matrix by restricting the sum of all entries in each row and in each column of the scramble matrix to be one. How to construct these rows in the constraint matrix is left as an exercise to the reader. :-)

This gives an additional $2\times4wh$ constraints on the puzzle, where $4wh$ is the number of rows (and columns) in the scramble matrix. However, these constraints are not all linearly independent. If we know all but one of these constraints, it is possible to deduce the final constraint. Therefore, the number of constraints we can add to the constraint matrix is $8wh - 1$.

#### Constraints on piece movement
As mentioned previosly, each $4\times4$ section of the scramble matrix can only take the form $\mathbf{R}_0$, $\mathbf{R}_1$, $\mathbf{R}_2$, $\mathbf{R}_3$ or $\mathbf{\phi}$. Given the following section of the transformation matrix
$$
    \begin{pmatrix}
        T_{1,1} & T_{1,2} & T_{1,3} & T_{1,4} \\
        T_{2,1} & T_{2,2} & T_{2,3} & T_{2,4} \\
        T_{3,1} & T_{3,2} & T_{3,3} & T_{3,4} \\
        T_{4,1} & T_{4,2} & T_{4,3} & T_{4,4}
    \end{pmatrix}
$$
this constraint can be added by requiring that
$$
    \begin{matrix}
        T_{1,1} - T_{2,2} = 0, & T_{2,2} - T_{3,3} = 0, & T_{3,3} - T_{4,4} = 0, & T_{4,4} - T_{1,1} = 0 \\
        T_{2,1} - T_{3,2} = 0, & T_{3,2} - T_{4,3} = 0, & T_{4,3} - T_{1,4} = 0, & T_{1,4} - T_{2,1} = 0 \\
        T_{3,1} - T_{4,2} = 0, & T_{4,2} - T_{1,3} = 0, & T_{1,3} - T_{2,4} = 0, & T_{2,4} - T_{3,1} = 0 \\
        T_{4,1} - T_{1,2} = 0, & T_{1,2} - T_{2,3} = 0, & T_{2,3} - T_{3,4} = 0, & T_{3,4} - T_{4,1} = 0
    \end{matrix}
$$
However, these equations are not fully linearly independent, as can be seen by
$$
    \left(T_{1,1} - T_{2,2}\right) + \left(T_{2,2} - T_{3,3}\right) + \left(T_{3,3} - T_{4,4}\right) = T_{1,1} - T_{4,4} = 0
$$
and similarly for the three other rows in the above system of equations. This means that for every four entries in the scramble matrix, there are three linearly independent constraints. This gives a total number of constraints of this type at $12w^2h^2$.

### Sum of constraints
In total, before any pieces are placed, the total number of constraints $n_c$ is therefore
$$
    \begin{align}
        n_c &= (2wh - w - h) + (8wh - 1) + 12w^2h^2 \\
        n_c &= 12w^2h^2 + 10wh - w - h - 1
    \end{align}
$$
There are $16w^2h^2$ entries in the scramble matrix, meaning that the system has fewer constraints than unknowns for every puzzle larger than $2\times1$.

### Locking in puzzle pieces
Locking in a puzzle piece in the scramble matrix means setting a $4\times4$ section of the matrix to one of $\mathbf{R}_0$, $\mathbf{R}_1$, $\mathbf{R}_2$, or $\mathbf{R}_3$, and represents locking in that a certain piece should be moved to a certain place and given a certain orientation. Doing this means that every other entry in the rows and columns affected in the scramble matrix also have their values locked to 0. This means that locking a piece in place constrains the value of $2\times4\times4wh - 16 = 32wh - 16$ entries in the scramble matrix. (The $-16$ term is to avoid double counting the constraints on the entries at the intersections of the rows and columns that are being constrained). However, the previous constraints on the entries in those rows and columns are now redundant, as are the constraints on the sums of the rows and columns. This means that $\frac{3}{4}(2\times4\times4wh-16) = 24wh - 12$ constraints on all the entries in the rows and columns can be removed, and $8$ constraints on the sums of the rows and columns affected. If the piece is placed next to one or more neigbouring pieces, there may also be some constraints due to connection shapes that are now redundant, but we'll ignore that for now in order to obtain a maximum bound on the number of constraints. This means that the sum of added constraints on matrix entries when placing the first piece is $8wh - 4$, while the sum of removed constraints on the rows and columns of the matrix is $8$.

However, care must be taken when placing subsequent puzzle pieces, as each subsequent locked row and column will intersect with each of the previously locked rows and columns. This means that for each subsequent locked piece, we must avoid double counting the matrix entries that were already locked when placing a previous piece. This requires a function of the number of placed pieces $n_p$ that takes the form
$$
    f(n_p) = f(n_p-1) + (8wh - 4) - 2\times4(n-1)
$$
where the term $f(n_p-1)$ is the number of entries that were already locked in by placing the previous puzzle piece, $(4wh+4)$ are the additional locked entries from placing piece number $n$ and $2\times4(n-1)$ is the number of entries that must not be double counted due to intersections with previously locked rows and columns. The following function satisifies the above equation:
$$
    f(n_p) = 8whn - 4n^2
$$

This gives an upper bound for the number of constraints after adding n pieces as
$$
    n_c(n_p) = 12w^2h^2 + 10wh - w - h - 1 + 8whn - 4n^2 - 8n
$$
(where the final $8n$ term is to account for the constraints on the sums of rows and columns that are made redundant by each piece placed).

In order for the scramble matrix to be determined, the number of constraints must be larger than the number of entries in the matrix:
$$
    \begin{align}
        16w^2h^2 &< 12w^2h^2 + 10wh - w - h - 1 + 8whn_p - 4n_p^2 - 8n_p \\
        0 &< -4n_p^2 + (8wh - 8)n_p -4w^2h^2 + 10wh - w - h - 1
    \end{align}
$$
This is a quadratic equation, which can be easily solved. The following table shows the number of pieces that must be fixed before the scramble matrix only has a single solution for various sizes of puzzle:

| Size of puzzle | Necessary number of pieces before system is overdetermined | Total pieces |
| -------------- | ---------------------------------------------------------- | ------------ |
| $5\times5$     | 20.72                                                      | 25           |
| $5\times6$     | 25.39                                                      | 30           |
| $6\times6$     | 31.03                                                      | 36           |
| $6\times7$     | 36.70                                                      | 42           |
| $7\times7$     | 43.34                                                      | 49           |
| $10\times10$   | 92.24                                                      | 100          |
| $100\times100$ | 9928.64                                                    | 10 000       |

As can be seen from the table, you need to have so many pieces already in placed before the puzzle unambigously solved that I don't think there is much to gain from trying to solve it using linear algebra. Unless, of course, there are more constraints that I have forgotten to include, which might bring the number of necessary pieces down.

In [ ]:
%load src/solution_finder.py

The following cell contains code to draw the puzzle

In [ ]:
%load src/puzzle_drawer.py

# Finding puzzles
Here, we find the verification matrix for a given width and height. Then we generate random scrambles until one is found where no two sides that were adjacent in the original puzzle are adjacent after the scramble. We then find a puzzle that is solved in both its unscrambled state and the state that results from the scramble we found. We then check that this puzzle doesn't have any connections that are repeated more than the maximum. If it does not, we check that it does not have any pieces that are duplicates of each other (i.e. have the same connections in the same order). If the puzzle meets all these conditions, we try to find all solutions for it. If the puzzle has only two solutions, it is returned, and a figure is drawn.

This is doable for $5\times5$ puzzles, seems to work for $5\times6$ puzzles, and seems to get completely bogged down for $6\times6$ puzzles. I think the solution finder needs to be made much more efficient in order to handle those.

I also suspect that the best way to generate puzzles more quickly is to find some smarter way to generate the scrambles, rather than generating them randomly and checking if they meet our criteria.

Rerun all the code in this notebook to see an illustration of a puzzle with only two solutions. With the default settings, this should happen almost immediately. Lowering the value of maximum_repeated_shapes should increase the runtime, but give more satisfying puzzles with fewer connection repetitions. Increasing the value of width and height should increase the runtime drastically. I've never seen it complete for a $6\times6$ puzzle so far.

In [ ]:
%load -r 8: src/main.py